In [31]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 29 09:31:48 2018

@author: eduardo-ssr
"""
import sys
import serial
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
from drawnow import drawnow
from time import sleep
from datetime import datetime

%matplotlib inline

read0 = []
read1 = []
read2 = []
read3 = []
read4 = []
read5 = []
read6 = []
read7 = []

dt = []
acelX = []
acelY = []
acelZ = []
gyroX = []
gyroY = []
gyroZ = []

constant_Calib_Acel = (16384/9.81)
constant_Calib_Gyro = 131


os.system('clear')
print ("======= Lista de dispositivos USB =======")
serial_ports = glob.glob('/dev/ttyUSB*')
while(len(serial_ports)==0):
    print ("Conecte o Arduino...\n")
    sleep(5);
    serial_ports = glob.glob('/dev/ttyUSB*')
        
for i in range(len(serial_ports)): 
    print (i, " - ", serial_ports[i])
    port = input("Escolha a porta do Arduino (e.g. 0): ")
        
ser = serial.Serial(serial_ports[int(port)], 115200,timeout=1)

def main():
    global contador
    global t
    global constant_Calib_Acel
    global constant_Calib_Gyro
    
    if ser.is_open:
        print("Comunicação Serial Estabelecida.\n")
    else :
        print("Erro na Comunicação Serial.\n")
        
        
    input("Pressione 'Enter' para iniciar a leitura e \"Ctrl+C\" para pausar: \n")
    
    
    
    while(True):    
        try:
            
            line = ser.readline().decode("utf-8")
            
            
            print(line)
            try:
                entry = line.split("\t")
                    
                
                dT = np.float(entry[0])
                AcX = np.float(entry[1])
                AcY = np.float(entry[2])
                AcZ = np.float(entry[3])
                temp = np.float(entry[4])
                Gx = np.float(entry[5])
                Gy = np.float(entry[6])
                Gz = np.float(entry[7])
                
                ACX = AcX/constant_Calib_Acel  # entre--20 m/s² e + 20 m/s² 
                ACY = AcY/constant_Calib_Acel  # entre--20 m/s² e + 20 m/s² 
                ACZ = AcZ/constant_Calib_Acel  # entre--20 m/s² e + 20 m/s² 
                
                Temp = temp/340.00 + 36.53
                
                GX = Gx/constant_Calib_Gyro # entre +250º/s e -250º/s
                GY = Gy/constant_Calib_Gyro # entre +250º/s e -250º/s
                GZ = Gz/constant_Calib_Gyro # entre +250º/s e -250º/s
                
                read0.append(dT)
                read1.append(AcX) 
                read2.append(AcY)
                read3.append(AcZ)
                read4.append(Gx)
                read5.append(Gy)
                read6.append(Gz)
                read7.append(temp)
                
                dt.append(dT)
                acelX.append(ACX)
                acelY.append(ACY)
                acelZ.append(ACZ)
                gyroX.append(GX)
                gyroY.append(GY)
                gyroZ.append(GZ)
                
                
                
            except (ValueError):
                print("Erro de valor.")
                pass
        
        except (KeyboardInterrupt):
            now = datetime.now()
            print ("Voce pressionou Ctrl+C para interromper este programa! Seus dados foram salvos em 'Dados_%s.csv'"%(str(now)[:-7]))

            ser.close()
            vX = []
            vX = np.diff(acelX)
            print(vX)
            
            fig, ax = plt.subplots(nrows = 1, ncols = 2, sharex=True, figsize=(15,5))
            
            ax[0].plot(acelX, "-r", label = 'Acel X')
            ax[0].plot(acelY, "-g", label = 'Acel Y')
            ax[0].plot(acelZ, "-b", label = 'Acel Z')
            ax[0].plot(vX, "r", label = 'Vel X')
            ax[0].set_xlabel('Tempo (ms)');
            ax[0].set_ylabel('Aceleração (m/s²)');
            ax[0].set_title('Dados de Aceleralção Linear')
            ax[0].legend(loc = 'lower left')
            
            ax[1].plot(gyroX, "-r", label = 'Gyro X' )
            ax[1].plot(gyroY, "-g", label = 'Gyro Y')
            ax[1].plot(gyroZ, "-b", label = 'Gyro Z')
            ax[1].set_xlabel('Tempo (ms)');
            ax[1].set_ylabel('Velocidade Angular (°/s)');
            ax[1].set_title('Dados de Velocidade Angular')
            ax[1].legend(loc = 'lower left')
            
            plt.show()
            
            x = np.vstack((read0,read1,read2,read3,read4,read5,read6,read7))
            y = np.vstack((dt,acelX,acelY,acelZ,gyroX,gyroY,gyroZ))
        
            np.savetxt('Dados_Brutos_%s.csv'%(str(now)[:-7]), np.transpose(x), delimiter=';')
            np.savetxt('Dados_Fisicos_%s.csv'%(str(now)[:-7]), np.transpose(y), delimiter=';') 
            break
if __name__ == "__main__":
    main()


======= Lista de dispositivos USB =======
0  -  /dev/ttyUSB0
Escolha a porta do Arduino (e.g. 0): 0
Comunicação Serial Estabelecida.

Pressione 'Enter' para iniciar a leitura e "Ctrl+C" para pausar: 

4.00	135.00	-210.00	-95.00



IndexError: list index out of range